In [ ]:
%matplotlib inline
'''
How to Optimize Your Dragon: V1
Import initialized model and start the optimization
'''
# system imports
import sys
import os
import datetime
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../..'))

import numpy as np
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import dill

# pyomo imports
from pyomo import environ as pe
from global_sets.component import m

from stages.reactive_stage import reactive_stage_rule
from stages.condenser_stage import condenser_stage_rule
from stages.reboiler_stage import reboiler_stage_rule

from utility.display_utility import beautify, beautify_reactive, HiddenLogs, HiddenPrints, plot_distribution,\
                                    trans_cnumber, trans_product_mole, check_product_spec
from utility.model_utility import add_dual, update_dual, delete_dual, check_DOF, tray_translator, check_iteration# , check_violate_constraint
from utility.data_utility import cal_cnumber
from utility.time_utility import create_filename_time, log_now, log_end

In [ ]:
logname = create_filename_time()
log_text_dir = './log/text/opt_'+logname+'.dat'
log_figure_dir = './log/figure/opt_'+logname+'.pdf'

In [ ]:
with open('./log/model/stage_20_base_product.pickle','rb') as f:
    model = dill.load(f)

In [ ]:
check_DOF(pe,model)

# Add tray optimization related sets and variables

In [ ]:
model.TRAY_total = pe.Set(initialize=['condenser']+[str(i) for i in model.TRAY]+['reboiler'])
model.TRAY_total.ordered = True

In [ ]:
model.P_tray_ratio = pe.Var(model.TRAY_total,m.PRODUCT,within=pe.NonNegativeReals,initialize=0,bounds=(0,1))
model.P_total = pe.Var(m.PRODUCT,within=pe.NonNegativeReals,initialize=0)
model.P_total_dry = pe.Var(m.PRODUCT,within=pe.NonNegativeReals,initialize=0)
model.x_P = pe.Var(m.COMP_TOTAL,m.PRODUCT,within=pe.NonNegativeReals,bounds=(0,1))
model.x_P_dry = pe.Var(m.COMP_ORG,m.PRODUCT,within=pe.NonNegativeReals,bounds=(0,1))

# Add equations

### Product split and mixing

In [ ]:
# sum of liquid draw for all products for each stage
def stage_sum_product_rule(model,j):
    return 1 == sum(model.P_tray_ratio[j,p] for p in m.PRODUCT)
model.stage_sum_product_con = pe.Constraint(model.TRAY_total,rule=stage_sum_product_rule)

# liquid product mass balance
def product_sum_stage_rule(model,p):
    return model.P_total[p] == sum(model.P_tray_ratio[j,p]*tray_translator(model,j).L['P'] for j in model.TRAY_total)
model.product_sum_stage_con = pe.Constraint(m.PRODUCT,rule=product_sum_stage_rule)

# liquid product component mass balance
def mass_balance_product_rule(model,i,p):
    return sum(model.P_tray_ratio[j,p]*tray_translator(model,j).L['P']*tray_translator(model,j).x[i] for j in model.TRAY_total) == model.P_total[p]*model.x_P[i,p]
model.mass_balance_product_con = pe.Constraint(m.COMP_TOTAL,m.PRODUCT,rule=mass_balance_product_rule)

### Dry composition

In [ ]:
# dry liquid product component 
def product_sum_dry_rule(model,p):
    return model.P_total_dry[p] == model.P_total[p] * (1 - sum(model.x_P[i,p] for i in m.COMP_INORG))
model.product_sum_dry_con = pe.Constraint(m.PRODUCT,rule=product_sum_dry_rule)

# dry liquid product component mass balance
def mass_balance_dry_rule(model,i,p):
    return model.x_P_dry[i,p] * (1 - sum(model.x_P[i,p] for i in m.COMP_INORG)) == model.x_P[i,p]
model.mass_balance_dry_con = pe.Constraint(m.COMP_ORG,m.PRODUCT,rule=mass_balance_dry_rule)

# Transfer from original liquid side-draw variable to product variable

### Transfer fixed side-draw variables

In [ ]:
model.P_tray_ratio[:,:].fix(0)
model.P_tray_ratio[:,'intermediate'].unfix();
model.P_tray_ratio['condenser','naphtha'].fix(1)
# model.P_tray_ratio['4','intermediate'].fix(1)
model.P_tray_ratio['7','gasoline'].fix(1)
model.P_tray_ratio['10','diesel'].fix(1)
model.P_tray_ratio['reboiler','heavy'].fix(1)

### Initialized newly created value

In [ ]:
model.P_total['naphtha'].set_value(model.condenser.L['P'].value)
model.P_total['intermediate'].set_value(model.reactive[4].L['P'].value)
model.P_total['gasoline'].set_value(model.reactive[7].L['P'].value)
model.P_total['diesel'].set_value(model.reactive[10].L['P'].value)
model.P_total['heavy'].set_value(model.reboiler.L['P'].value)

for i in m.COMP_TOTAL:
    model.x_P[i,'naphtha'].set_value(model.condenser.x[i].value)
    model.x_P[i,'intermediate'].set_value(model.reactive[4].x[i].value)
    model.x_P[i,'gasoline'].set_value(model.reactive[7].x[i].value)
    model.x_P[i,'diesel'].set_value(model.reactive[10].x[i].value)
    model.x_P[i,'heavy'].set_value(model.reboiler.x[i].value)

In [ ]:
model.P_total_dry['naphtha'].set_value(model.condenser.L['P'].value * (1 - sum(model.condenser.x[i].value for i in m.COMP_INORG)))
model.P_total_dry['intermediate'].set_value(model.reactive[4].L['P'].value * (1 - sum(model.reactive[4].x[i].value for i in m.COMP_INORG)))
model.P_total_dry['gasoline'].set_value(model.reactive[7].L['P'].value * (1 - sum(model.reactive[7].x[i].value for i in m.COMP_INORG)))
model.P_total_dry['diesel'].set_value(model.reactive[10].L['P'].value * (1 - sum(model.reactive[10].x[i].value for i in m.COMP_INORG)))
model.P_total_dry['heavy'].set_value(model.reboiler.L['P'].value * (1 - sum(model.reboiler.x[i].value for i in m.COMP_INORG)))

for i in m.COMP_ORG:
    model.x_P_dry[i,'naphtha'].set_value(model.condenser.x[i].value / (1 - sum(model.condenser.x[i].value for i in m.COMP_INORG)))
    model.x_P_dry[i,'intermediate'].set_value(model.reactive[4].x[i].value / (1 - sum(model.reactive[4].x[i].value for i in m.COMP_INORG)))
    model.x_P_dry[i,'gasoline'].set_value(model.reactive[7].x[i].value / (1 - sum(model.reactive[7].x[i].value for i in m.COMP_INORG)))
    model.x_P_dry[i,'diesel'].set_value(model.reactive[10].x[i].value / (1 - sum(model.reactive[10].x[i].value for i in m.COMP_INORG)))
    model.x_P_dry[i,'heavy'].set_value(model.reboiler.x[i].value / (1 - sum(model.reboiler.x[i].value for i in m.COMP_INORG)))

### Initialize

In [ ]:
check_DOF(pe,model)

In [ ]:
opt = pe.SolverFactory('ipopt')

opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma86'
opt.options['output_file'] = './tmp/ipopt_output_tmp.output'
opt.options['linear_system_scaling '] = 'mc19'
opt.options['linear_scaling_on_demand '] = 'no'

opt.options['max_iter'] = 7000
opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-20
opt.options['warm_start_mult_bound_push'] = 1e-20
opt.options['mu_init'] = 1e-6

In [ ]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

In [ ]:
pdf = PdfPages(log_figure_dir)

In [ ]:
with HiddenLogs(log_text_dir,'w'):
    print('\n>','Original 20 stage case')
    print('-'*108)
    beautify(pe,model)
    check_product_spec(model)
    log_now()

plot_distribution(model,pdf,'Original 20 stage case')

# Optimization Input

**Parameters:**
* Stage Temperature
    * Rectifying section: 200C - 300C
    * Stripping section: 200C - 300C
* Reflux
    * Distillate / (Distillate + Reflux): 0.5 - 0.05, Refulx Ratio: 1 - 19
* Side-draw ratio
    * PR_L: 0 - 1

**Constraints:**
* Product
    * Distillate: C5~C7 >= 0.75
    * Gasoline: C8~C12 >= 0.75
    * Diesel: C13~C18 >= 0.6
    * Heavy: C19+ >= 0.85
    
**Objective:**
* Max gasoline production

### Open up parameters

In [ ]:
# temperature
for j in model.TRAY_reactive:
    model.reactive[j].T.setlb(200+273.15)
    model.reactive[j].T.setub(300+273.15)

# reflux
model.condenser.PR_L.unfix()
model.condenser.PR_L.setlb(0.05)
model.condenser.PR_L.setub(0.5)

# stage side-draw
model.reactive[4].PR_L.unfix()
model.reactive[7].PR_L.unfix()
model.reactive[10].PR_L.unfix()

### Constraints

In [ ]:
model.quality_spec = pe.Param(m.PRODUCT,initialize={\
                    'naphtha':0.75,'gasoline':0.75,'diesel':0.6,'heavy':0.85},mutable=True)

In [ ]:
def product_spec_rule(model,p):
    if p == 'intermediate':
        return pe.Constraint.Skip
    return sum(model.x_P_dry[i,p] for i in m.PRODUCT_cnumber[p]) >= model.quality_spec[p]
model.product_spec_con = pe.Constraint(m.PRODUCT,rule=product_spec_rule)

### Objective

In [ ]:
model.del_component(model.obj)
model.obj = pe.Objective(expr = model.P_total['gasoline'] - sum(model.reactive[j].MPCC.pf for j in model.reactive)\
                                - model.reboiler.MPCC.pf, sense = pe.maximize)

In [ ]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

In [ ]:
with HiddenLogs(log_text_dir):
    print('\n>','Optimized Product Side Draw')
    print('-'*108)
    beautify(pe,model)
    check_product_spec(model)
    log_now()

plot_distribution(model,pdf,'Optimized Temperature, Reflux and Product Side Draw')